In [ ]:
import os
from unidecode import unidecode
import pandas as pd
import numpy as np
from great_tables import GT, html
import warnings
warnings.filterwarnings('ignore')

In [ ]:
## MI variable
my_variable = int(os.getenv("MY_VARIABLE"))

In [ ]:
metadatos = pd.read_csv("../metadata/metadatos_espacios.csv").iloc[my_variable,:]

In [ ]:
# FUNCIONES Y ENLACES
# URL de Biota
url_biota = "https://www.biodiversidadcanarias.es/biota/especie/"

def create_gt(df, enp, categoria, n_species):
    """_Resumen:
    Esta función tiene como objetivo procesar los datos de las tablas CSV en el
    directorio ENP (Espacios Naturales Protegidos), con la intención de crear 
    una GREAT TABLE con las especies observadas en un espacio en concreto.

    Args:
        df (DataFrame): Data Frame de uno de los ENP
        enp (string): ENP seleccionado
        categoria (string): Categoría del ENP seleccionado

    Returns:
        great_table: Tabla HTML GT
    """
    df.id_biota = "[" + df.id_biota + "]" + "(" + url_biota + df.id_biota + ")"

    df = df.sort_values(["reino", "especie"])

    df['nombre_comun'] = np.where(
        (df['nombre_comun']
            .str.replace(",", "").
            str.split().
            str.len() > 4),
        (df['nombre_comun']
            .str.replace(",", "")
            .str.split()
            .str[:4]
            .str.join(', ')) + ", etc",
        df['nombre_comun']
    )

    gt = (GT(df)
        .tab_header(
            title = html(
                f"<strong>INVENTARIO DE ESPECIES DEL ESPACIO NATURAL PROTEGIDO (ENP) {enp}, ISLA DE GRAN CANARIA</strong>"
            ),
            subtitle = html(f"Categoría del ENP: <u>{categoria}</u><br><i>Número de especies en total del ENP = <strong>{n_species}</strong><i>")
        )
        .cols_label(
            id_biota = html("<strong>ID Biota</strong>"),
            especie = html("<strong>Nombre científico</strong>"),
            nombre_comun = html("<strong>Nombre común</strong>"),
            endemicidad = html("<strong>Endemicidad</strong>"),
            reino = html("<strong>Reino</strong>"),
            group_count = html("<strong>Nº fotos</strong>")
        )
        .fmt_markdown(columns = "id_biota")
        .opt_stylize()
        .tab_options(
            heading_border_bottom_color = "black",
            table_border_top_color="black", 
            table_border_bottom_color="black",
            column_labels_border_top_color = "black",
            column_labels_border_bottom_color = "black",
            table_body_border_bottom_color = "black"

        ))
    return gt

In [ ]:
## Leemos los datos usando metadata/ para automatizar el proceso
df_seleccionado = pd.read_csv(
    f"../ENP/{unidecode(metadatos[3].lower().replace(' ', '_'))}/{metadatos[2]}"
)
logo_enp = f"<a target='_blank' rel='noreferrer'> <img src='{metadatos[4]}' alt='logoenp' width='30' height='30'/> </a>"
## Nº de especies en total
n_especies = df_seleccionado.shape[0]

In [ ]:
df_seleccionado_table = create_gt(df=df_seleccionado, 
                                  enp=f"{metadatos[0] + " " + metadatos[1].upper()}", 
                                  categoria=f"{metadatos[3] + logo_enp}",
                                  n_species=n_especies)

In [ ]:
df_seleccionado_table

<footer style='text-align: left; margin-top: 20px; font-size: small; color: gray;'>
    <p>Sitio <a href='https://github.com/JuanCarlosBio/inventario_especies_enp'>desarrollado</a> por <a href='https://juancarlosbio.github.io/juancarlos_portfolio_esp/'>Juan Carlos García Estupiñán</a> mediante la MIT LICENSE</p>
</footer>